In [ ]:
!pip install transformers torch pillow donut-python
!pip install transformers==4.25.1
!pip install pytorch-lightning==1.6.4
!pip install timm==0.5.4
!pip install gradio

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Preparing metadata (setup

In [ ]:
from google.colab import files
from PIL import Image

uploaded = files.upload()  # Allows you to upload the image
image_path = list(uploaded.keys())[0]  # Get the uploaded file's name

Saving testing.png to testing.png


In [ ]:
import torch
from PIL import Image
import numpy as np
from io import BytesIO

from donut import DonutModel

class DonutInference:
    def __init__(self, task_name="cord-v2", pretrained_path="naver-clova-ix/donut-base-finetuned-cord-v2"):
        self.task_name = task_name
        self.task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>" if "docvqa" == task_name else f"<s_{task_name}>"
        self.pretrained_model = self._load_model(pretrained_path)

    def _load_model(self, pretrained_path):
        model = DonutModel.from_pretrained(pretrained_path)
        if torch.cuda.is_available():
            model.half()
            model.to(torch.device("cuda"))
        model.eval()
        return model

    def run_inference(self, image):
        # Ensure image is in PIL.Image.Image format
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        elif isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        elif isinstance(image, BytesIO):
            image = Image.open(image).convert("RGB")

        output = self.pretrained_model.inference(image=image, prompt=self.task_prompt)["predictions"][0]
        return output


if __name__ == "__main__":
    # Use the uploaded image path
    image = Image.open(image_path).convert("RGB")
    image.show()
    donut = DonutInference()
    print(donut.run_inference(image))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/404 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/859M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

{'menu': [{'nm': 'ICE BLAOKCOFFE', 'cnt': '2', 'price': '82,000'}, {'nm': 'AVOCADO COFFEE', 'cnt': '1', 'price': '61,000'}, {'nm': 'Oud CHINEN KATSU FF', 'cnt': '1', 'price': '51,000'}], 'sub_total': {'subtotal_price': '194,000', 'discount_price': '19,400'}, 'total': {'total_price': '174,600', 'cashprice': '200,000', 'changeprice': '25,400'}}
